# eXamine Automation Tutorial

This case study demonstrates how to use the REST API of eXamine to study a small, annotated graph in Cytoscape. The graph we study is `Zachary's karate club`.

In [30]:
# HTTP Client for Python
import requests
# Standard JSON library, currently unused
import json

# Cytoscape port number
PORT_NUMBER = 1234

BASE_URL = "file:///Users/melkebir/Projects/eXamine/doc/tutorial/"

# The Base path for the CyRest API
BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'

#Helper command to call a command via HTTP GET
def executeRestCommand(namespace="", command="", args={}):
    postString = BASE + "commands/" + namespace + "/" + command
    firstarg = True;
    for arg in args:
        postString += ("?" if firstarg else "&")
        postString += arg+"="+args[arg]
        if (firstarg):
            firstarg = False;

    res = requests.get(postString)
    return res

## Importing network and node-specific annotation

We start by importing the graph directly from the eXamine repository on github.

In [31]:
# First we import our demo network
res = executeRestCommand("network", "import url", {"indexColumnSourceInteraction":"1",
                                                   "indexColumnTargetInteraction":"2",
                                                   "url": BASE_URL + "edges_karate.gml"})

Next, we identify the SUID of the newly created network. The SUID is needed for subsequent eXamine commands.

In [32]:
networkSUID = res.text.split("suid=",1)[1].split(",")[0];

We then import node-specific annotation directly from the eXamine repository on github. The imported file contains set membership information for each node. Note that it is important to ensure that set-membership information is imported as `List of String`, as indicated by `sl`.

In [33]:
# Next we import node annotations
executeRestCommand("table", "import url",
                   {"firstRowAsColumnNames":"true",
                    "keyColumnIndex" : "1",
                    "startLoadRow" : "1",
                    "dataTypeList":"s,sl",
                    "url": BASE_URL + "nodes_karate.txt"})

<Response [200]>

In [34]:
executeRestCommand("network", "select", {"nodeList" : "all"})

<Response [200]>

In [35]:
executeRestCommand("examine", "generate groups",
                   {"selectedGroupColumns" : "Community"})

<Response [200]>

## Set-based visualization using eXamine

We now describe how to visualize the current selection. First, we set the visualization options.

In [39]:
# Adjust the visualization settings
executeRestCommand("examine", "update settings",
                   {"labelColumn" : "label",
                    "URL" : "label",
                    "showScore" : "false",
                    "selectedGroupColumns" : "Community"})

<Response [404]>

We then select six groups.

In [37]:
# Select groups for demarcation in the visualization
executeRestCommand("examine", "select groups",
                   {"selectedGroups":"A,B,C,D,E,F"})

<Response [200]>

There are two options: either we launch the interactive eXamine visualization, or we directly generate an SVG.

In [12]:
# Launch the interactive eXamine visualization
executeRestCommand("examine", "interact", {})

<Response [200]>

The command below launches the eXamine window. If this window is blank, simply resize the window to force a redraw of the scene.

In [40]:
# Export a graphic instead of interacting with it
# use absolute path; writes in Cytoscape directory if not changed 
executeRestCommand("examine", "export", {"path": "test.svg"})

<Response [200]>